In [1]:
import numpy as np
import os, sys, cv2, time
import matplotlib.pyplot as plt
import mxnet as mx
from mxnet import gluon, init, autograd, nd, image
from mxnet.gluon import data as gdata, utils as gutils, nn, loss as gloss
from mxnet.gluon.data.vision import datasets, transforms
from sklearn.metrics import confusion_matrix
from ipywidgets import interact, IntSlider

C:\Users\lab\AppData\Local\Continuum\anaconda3\envs\mxnet-gpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#target_folder = 'E:/Guan-Ming/Deep_ML/all_data_resize/'
#output_folder = 'E:/Guan-Ming/Deep_ML/all_data_final_transposed/'
#test_data_folder = 'E:/Guan-Ming/Deep_ML/test_data_resize/'
#extra_validation_target_folder = 'E:/Guan-Ming/Deep_ML/extra_validation_data_resize/'

test_data_folder = 'Test_Data/'
extra_validation_target_folder = 'Extra_Validation_Data/'
final_validation_target_folder = 'Scoring_Validation_Data/'
fine_tuning_data_folder = 'Fine_Tuning_Data/'
#fine_tuning_data_folder = 'Fine_Tuning_Data_with_Extra_Validation/'

### Read in Fine-Tuning Data

In [3]:
files = os.listdir(fine_tuning_data_folder)
total_fine_tuning_data = len(files)
fine_tuning_data_x = nd.zeros((total_fine_tuning_data, 256, 256, 3)).astype(np.uint8)
fine_tuning_data_y = nd.zeros(total_fine_tuning_data)
total_num = 0

for index, f in enumerate(files):
    fine_tuning_data_x[index] = image.imread(fine_tuning_data_folder+f)

    label = f.split('_',1)[0]
    if label =='0':
        fine_tuning_data_y[index] = 0
        total_num += 1
    elif label =='1':
        fine_tuning_data_y[index] = 1
        total_num += 1
    elif label =='2':
        fine_tuning_data_y[index] = 2
        total_num += 1
    elif label =='3':
        fine_tuning_data_y[index] = 3
        total_num += 1
    elif label =='4':
        fine_tuning_data_y[index] = 4
        total_num += 1
    elif label =='5':
        fine_tuning_data_y[index] = 5
        total_num += 1
    else:
        raise RuntimeError('Cannot label training data!')
#    break

if total_num == total_fine_tuning_data:
    print("Sum check pass!")

Sum check pass!


### Read in Testing Data

In [4]:
files = os.listdir(test_data_folder)
total_test_data = len(files)
test_data_x = nd.zeros((total_test_data, 256, 256, 3)).astype(np.uint8)
test_data_y = nd.zeros(total_test_data)
total_num = 0

for index in range(total_test_data):
    f = files[index]
    test_data_x[index] = image.imread(test_data_folder+f)
    label = f.split('_', 1)[0]
    if label == '0':
        test_data_y[index] = 0
        total_num += 1
    elif label == '1':
        test_data_y[index] = 1
        total_num += 1
    elif label == '2':
        test_data_y[index] = 2
        total_num += 1
    elif label == '3':
        test_data_y[index] = 3
        total_num += 1
    elif label == '4':
        test_data_y[index] = 4
        total_num += 1
    elif label == '5':
        test_data_y[index] = 5
        total_num += 1
    else:
        raise RuntimeError('Cannot label testation data!')
        
if total_num == total_test_data:
    print("Sum check pass!")

Sum check pass!


### Read in Extra Validation Data

In [5]:
files = os.listdir(extra_validation_target_folder)
total_extra_validation_data = len(files)
extra_validation_data_x = nd.zeros((total_extra_validation_data, 256, 256, 3)).astype(np.uint8)
extra_validation_data_y = nd.zeros(total_extra_validation_data)
total_num = 0

for index, f in enumerate(files):
    extra_validation_data_x[index] = image.imread(extra_validation_target_folder+f)

    label = f.split('_',1)[0]
    if label =='0':
        extra_validation_data_y[index] = 0
        total_num += 1
    elif label =='1':
        extra_validation_data_y[index] = 1
        total_num += 1
    elif label =='2':
        extra_validation_data_y[index] = 2
        total_num += 1
    elif label =='3':
        extra_validation_data_y[index] = 3
        total_num += 1
    elif label =='4':
        extra_validation_data_y[index] = 4
        total_num += 1
    elif label =='5':
        extra_validation_data_y[index] = 5
        total_num += 1
    else:
        raise RuntimeError('Cannot label training data!')
#    break

if total_num == total_extra_validation_data:
    print("Sum check pass!")

Sum check pass!


### Read in Final Validation Data Set

In [6]:
files = os.listdir(final_validation_target_folder)
total_final_validation_data = len(files)
final_validation_data_x = nd.zeros((total_final_validation_data, 256, 256, 3)).astype(np.uint8)
final_validation_data_y = nd.zeros(total_final_validation_data)
total_num = 0

for index, f in enumerate(files):
    final_validation_data_x[index] = image.imread(final_validation_target_folder+f)

    label = f.split('_',1)[0]
    if label =='0':
        final_validation_data_y[index] = 0
        total_num += 1
    elif label =='1':
        final_validation_data_y[index] = 1
        total_num += 1
    elif label =='2':
        final_validation_data_y[index] = 2
        total_num += 1
    elif label =='3':
        final_validation_data_y[index] = 3
        total_num += 1
    elif label =='4':
        final_validation_data_y[index] = 4
        total_num += 1
    elif label =='5':
        final_validation_data_y[index] = 5
        total_num += 1
    else:
        raise RuntimeError('Cannot label training data!')
#    break

if total_num == total_final_validation_data:
    print("Sum check pass!")

Sum check pass!


In [7]:
#train_mean = [0.592143940485608, 0.5347022953513785, 0.4981124834847486]  #over training data, train_size=0.65
#train_std = [0.2880660398985942, 0.2820819880842514, 0.282954693408029]  #over training data, train_size=0.65

train_mean = [0.5905187523955868, 0.5332879723913606, 0.49680592040529487]  #over all data
train_std = [0.28736647217403904, 0.28094815765222825, 0.2815392173263535]  #over all data

# extra_validation_mean = [0.48470740652217936, 0.4379602084974611, 0.37406493007969804]
# extra_validation_std = [0.2525134266173694, 0.23321502528915997, 0.2119784063910089]
# final_validation_mean = [0.56024206031756862, 0.51607103106300856, 0.48234928793204251]
# final_validation_std = [0.29965082389575942, 0.28886936891107284, 0.28323027528012074]

In [8]:
# train_mean_r = (fine_tuning_data_x[:,:,:,0].asnumpy()/255).mean()
# train_mean_g = (fine_tuning_data_x[:,:,:,1].asnumpy()/255).mean()
# train_mean_b = (fine_tuning_data_x[:,:,:,2].asnumpy()/255).mean()
# train_std_r = (fine_tuning_data_x[:,:,:,0].asnumpy()/255).std()
# train_std_g = (fine_tuning_data_x[:,:,:,1].asnumpy()/255).std()
# train_std_b = (fine_tuning_data_x[:,:,:,2].asnumpy()/255).std()
# train_mean = [train_mean_r, train_mean_g, train_mean_b]
# train_std = [train_std_r, train_std_g, train_std_b]
# train_mean, train_std

### Create Data Iterator

In [9]:
batch_size = 64
num_workers = 0 if sys.platform.startswith('win32') else 4

train_dataset = gdata.ArrayDataset(fine_tuning_data_x, fine_tuning_data_y)
test_dataset = gdata.ArrayDataset(test_data_x, test_data_y)
extra_validation_dataset = gdata.ArrayDataset(extra_validation_data_x, extra_validation_data_y)
final_validation_dataset = gdata.ArrayDataset(final_validation_data_x, final_validation_data_y)
    
transformer = []
transformer += [gdata.vision.transforms.ToTensor()] # transer the train data from shape (sample, H, W, channel) to (sample, channel, H, W) and rescale to between 0 and 1 
transformer += [gdata.vision.transforms.Normalize(train_mean, train_std)]
transformer = gdata.vision.transforms.Compose(transformer)

train_iter = gdata.DataLoader(train_dataset.transform_first(transformer), batch_size = batch_size, shuffle=True, num_workers=num_workers)
test_iter = gdata.DataLoader(test_dataset.transform_first(transformer), batch_size = batch_size, shuffle=False, num_workers=num_workers)
extra_validation_iter = gdata.DataLoader(extra_validation_dataset.transform_first(transformer), batch_size = batch_size, shuffle=False, num_workers=num_workers)
final_validation_iter = gdata.DataLoader(final_validation_dataset.transform_first(transformer), batch_size = batch_size, shuffle=False, num_workers=num_workers)

# extra_transformer = []
# extra_transformer += [gdata.vision.transforms.ToTensor()] # transer the train data from shape (sample, H, W, channel) to (sample, channel, H, W) and rescale to between 0 and 1 
# extra_transformer += [gdata.vision.transforms.Normalize(extra_validation_mean, extra_validation_std)]
# extra_transformer = gdata.vision.transforms.Compose(extra_transformer)
# extra_validation_iter = gdata.DataLoader(extra_validation_dataset.transform_first(extra_transformer), batch_size = batch_size, shuffle=False, num_workers=num_workers)

# final_transformer = []
# final_transformer += [gdata.vision.transforms.ToTensor()] # transer the train data from shape (sample, H, W, channel) to (sample, channel, H, W) and rescale to between 0 and 1 
# final_transformer += [gdata.vision.transforms.Normalize(final_validation_mean, final_validation_std)]
# final_transformer = gdata.vision.transforms.Compose(final_transformer)
# final_validation_iter = gdata.DataLoader(final_validation_dataset.transform_first(final_transformer), batch_size = batch_size, shuffle=False, num_workers=num_workers)

### Define Functions for Training

In [10]:
def try_gpu():
    """If GPU is available, return mx.gpu(0); else return mx.cpu()."""
    try:
        ctx = mx.gpu()
        _ = nd.array([0], ctx=ctx)
    except mx.base.MXNetError:
        ctx = mx.cpu()
    return ctx
def evaluate_accuracy(data_iter, net, ctx=[mx.cpu()]):
    """Evaluate accuracy of a model on the given data set."""
    if isinstance(ctx, mx.Context):
        ctx = [ctx]
    acc_sum, n = nd.array([0]), 0
    for batch in data_iter:
        features, labels, _ = _get_batch(batch, ctx)
        for X, y in zip(features, labels):
            y = y.astype('float32')
            acc_sum += (net(X).argmax(axis=1) == y).sum().copyto(mx.cpu())
            n += y.size
        acc_sum.wait_to_read()
    return acc_sum.asscalar() / n

def _get_batch(batch, ctx):
    """Return features and labels on ctx."""
    features, labels = batch
    if labels.dtype != features.dtype:
        labels = labels.astype(features.dtype)
    return (gutils.split_and_load(features, ctx),
            gutils.split_and_load(labels, ctx), features.shape[0])

def train(net, train_iter, test_iter, batch_size, trainer, ctx, num_epochs, early_stop = False, temp_tesy_acc = 0.0):
    """Train and evaluate a model with CPU or GPU."""
    print('training on', ctx)
    loss = gloss.SoftmaxCrossEntropyLoss()
    average_time = 0.0
    
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        if early_stop:
            net.save_parameters('./temp_net_weight.params')
        for X, y in train_iter:
            X, y = X.as_in_context(ctx), y.as_in_context(ctx)
            with autograd.record():
                y_hat = net(X)            
                l = loss(y_hat, y).sum()
            l.backward()
            trainer.step(batch_size)
            y = y.astype('float32')
            train_l_sum += l.asscalar()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().asscalar()
            n += y.size
        
        test_acc = evaluate_accuracy(test_iter, net, ctx)
        epoch_time = time.time() - start
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, '
              'time %.1f sec'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc, epoch_time))
        average_time += epoch_time
        
        if early_stop:
            if temp_tesy_acc>test_acc:
                break
            else:
                temp_tesy_acc = test_acc
    print('Average time per epoch is %.4f s.'%(average_time/(epoch+1)))

In [11]:
ctx = try_gpu()

### RESNET-34

In [12]:
class Residual(nn.Block):
    def __init__(self, num_channels, use_1x1conv=False, strides=1, **kwargs):
        super(Residual, self).__init__(**kwargs) 
        self.conv1 = nn.Conv2D(num_channels, kernel_size=3, padding=1, strides=strides) # this layer down-sampling the input
        self.conv2 = nn.Conv2D(num_channels, kernel_size=3, padding=1)  # the layer doesn't down-sampling the input
        
        if use_1x1conv:
            self.conv3 = nn.Conv2D(num_channels, kernel_size=1, strides=strides)
            # to match the dimension of feature map of conv1 + conv2, the strides must be set to be identicla to conv 1
        else: 
            self.conv3 = None 
            
        self.bn1 = nn.BatchNorm() 
        self.bn2 = nn.BatchNorm()
        
    def forward(self, X):
        Y = nd.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y)) 
        if self.conv3:
            X = self.conv3(X)

        return nd.relu(Y + X)
    
def resnet_block(num_channels, num_residuals, first_block=False): 
    blk = nn.Sequential() 
    for i in range(num_residuals): 
        if i == 0 and not first_block:
            #net.add(Residual(num_channels, use_1x1conv=True, strides=2))
            blk.add(Residual(num_channels, use_1x1conv=True, strides=2))
        else:
            #net.add(Residual(num_channels, strides=1))
            blk.add(Residual(num_channels, strides=1))
    return blk

In [23]:
num_of_output_channels = [64, 128, 256, 512]
num_residuals = [3, 4, 6, 3]

net_RESNET = nn.Sequential() 
net_RESNET.add(nn.Conv2D(64, kernel_size=7, strides=2, padding=3),
        nn.BatchNorm(), 
        nn.Activation('relu'),
        nn.MaxPool2D(pool_size=3, strides=2, padding=1))

for i, num in enumerate(num_of_output_channels):
    if i==0:
        net_RESNET.add(resnet_block(num, num_residuals[i], first_block=True))
    else:
        net_RESNET.add(resnet_block(num, num_residuals[i]))
    net_RESNET.add(nn.Dropout(0.1))

net_RESNET.add(nn.GlobalAvgPool2D(), nn.Dense(6))

### RESNET 38

In [24]:
net = resnet_block(1024, 3)
net.add(nn.Dropout(0.5))
net.add(nn.GlobalAvgPool2D(), nn.Dense(6))

In [25]:
net_RESNET_38 = nn.Sequential()
net_RESNET_38.add(net_RESNET[:12], net)
#net_RESNET_38.load_parameters('C:/Users/lab/Jupyter_Notebook/Guan-Ming/Midterm_Project/NN_params/train_test_mixed/RESNET38_DROPOUT_train_data_transposed_1_fine_tuned.params', ctx=ctx)

### Initialize or Load Weights

In [26]:
net_RESNET.load_parameters('C:/Users/lab/Jupyter_Notebook/Guan-Ming/Midterm_Project/NN_params/train_test_mixed/RESNET34_DROPOUT_train_data_transposed_4_fine_tuned.params', ctx= ctx)
net.initialize(ctx=ctx, init=mx.init.Xavier(rnd_type ='gaussian', factor_type='in', magnitude=2.0))
#net_RESNET.initialize(ctx=ctx, init=mx.init.Xavier(rnd_type ='gaussian', factor_type='in', magnitude=2.0))

In [27]:
for X,_ in train_iter:
    net_RESNET_38.summary(X.as_in_context(ctx))
    break

--------------------------------------------------------------------------------
        Layer (type)                                Output Shape         Param #
               Input                           (64, 3, 256, 256)               0
            Conv2D-1                          (64, 64, 128, 128)            9472
         BatchNorm-2                          (64, 64, 128, 128)             256
        Activation-3                          (64, 64, 128, 128)               0
         MaxPool2D-4                            (64, 64, 64, 64)               0
            Conv2D-5                            (64, 64, 64, 64)           36928
         BatchNorm-6                            (64, 64, 64, 64)             256
            Conv2D-7                            (64, 64, 64, 64)           36928
         BatchNorm-8                            (64, 64, 64, 64)             256
          Residual-9                            (64, 64, 64, 64)               0
           Conv2D-10        

In [36]:
#net_RESNET_38.load_parameters('temp_net_weight.params', allow_missing=True, ignore_extra=True)

In [36]:
net_RESNET_38.load_parameters('temp_net_weight.params', ctx=ctx)

In [40]:
adam_optimizer = mx.optimizer.Adam(learning_rate=0.01e-4, beta1=0.9, beta2=0.9)
trainer = gluon.Trainer(net_RESNET_38[1].collect_params(), optimizer=adam_optimizer)
train(net_RESNET_38, train_iter, final_validation_iter, batch_size, trainer, ctx, 20, True)

training on gpu(0)
epoch 1, loss 0.0438, train acc 0.992, test acc 0.928, time 3.2 sec
epoch 2, loss 0.0240, train acc 1.000, test acc 0.928, time 3.1 sec
epoch 3, loss 0.0383, train acc 0.992, test acc 0.928, time 3.2 sec
epoch 4, loss 0.0155, train acc 1.000, test acc 0.931, time 3.1 sec
epoch 5, loss 0.0474, train acc 0.992, test acc 0.926, time 3.1 sec
Average time per epoch is 3.1416 s.


### Print Accuracy for Testing Data Set

In [37]:
print('Accuracy of model is %.4f %% .'%(100*evaluate_accuracy(test_iter, net_RESNET, ctx)))
print('Accuracy of model is %.4f %% .'%(100*evaluate_accuracy(test_iter, net_RESNET_38, ctx)))

Accuracy of model is 99.3333 % .
Accuracy of model is 99.6667 % .


### Print Accuracy for Extra Validation Data Set 

In [38]:
print('Accuracy of model is %.4f %% .'%(100*evaluate_accuracy(extra_validation_iter, net_RESNET, ctx)))
print('Accuracy of model is %.4f %% .'%(100*evaluate_accuracy(extra_validation_iter, net_RESNET_38, ctx)))

Accuracy of model is 75.0000 % .
Accuracy of model is 73.2143 % .


### Print Accuracy of Final Scoring Data Set

In [39]:
print('Accuracy of model is %.4f %% .'%(100*evaluate_accuracy(final_validation_iter, net_RESNET, ctx)))
print('Accuracy of model is %.4f %% .'%(100*evaluate_accuracy(final_validation_iter, net_RESNET_38, ctx)))

Accuracy of model is 93.0864 % .
Accuracy of model is 93.0864 % .


### Data Saving and Network Structure

In [25]:
#net_RESNET.save_parameters('C:/Users/lab/Jupyter_Notebook/Guan-Ming/Midterm_Project/NN_params/train_test_mixed/RESNET34_DROPOUT_train_data_transposed_4_fine_tuned.params')
#===============================================================
# num_jitter = 3
# color_aug = gdata.vision.transforms.RandomColorJitter(brightness=0.3, contrast=0.6)
#===============================================================
# num_of_output_channels = [64, 128, 256, 512]
# num_residuals = [3, 4, 6, 3]

# net_RESNET = nn.Sequential() 
# net_RESNET.add(nn.Conv2D(64, kernel_size=7, strides=2, padding=3),
#         nn.BatchNorm(), 
#         nn.Activation('relu'),
#         nn.MaxPool2D(pool_size=3, strides=2, padding=1))

# for i, num in enumerate(num_of_output_channels):
#     if i==0:
#         net_RESNET.add(resnet_block(num, num_residuals[i], first_block=True))
#     else:
#         net_RESNET.add(resnet_block(num, num_residuals[i]))
#     net_RESNET.add(nn.Dropout(0.1))

# net_RESNET.add(nn.GlobalAvgPool2D(), nn.Dense(6))
#===============================================================

In [16]:
#net_RESNET_38.save_parameters('C:/Users/lab/Jupyter_Notebook/Guan-Ming/Midterm_Project/NN_params/train_test_mixed/RESNET38_DROPOUT_train_data_transposed_1_fine_tuned.params')
#===============================================================
# num_jitter = 3
# color_aug = gdata.vision.transforms.RandomColorJitter(brightness=0.3, contrast=0.6)
#===============================================================
#===============================================================

### Confusion Matrix of Testing Data Set

In [74]:
y_pred_test = np.zeros(total_test_data)
count = 0
for X, _ in test_iter:
    y_pred_test[count:min(count+batch_size, total_test_data)] = net_RESNET(X.as_in_context(ctx)).asnumpy().argmax(axis=1)
    count += batch_size
wrong_index_test = np.argwhere(y_pred_test!=test_data_y.asnumpy())

In [75]:
result = confusion_matrix(test_data_y.asnumpy(), y_pred_test)
print(result)
print('\n')
for i in range(6):
    print("Accuracy of %d is %.2f %%." %(i, 100*result[i][i]/result[i].sum()))

[[50  0  0  0  0  0]
 [ 0 49  1  0  0  0]
 [ 0  0 50  0  0  0]
 [ 0  0  0 50  0  0]
 [ 0  0  0  0 49  1]
 [ 0  0  0  0  1 49]]


Accuracy of 0 is 100.00 %.
Accuracy of 1 is 98.00 %.
Accuracy of 2 is 100.00 %.
Accuracy of 3 is 100.00 %.
Accuracy of 4 is 98.00 %.
Accuracy of 5 is 98.00 %.


In [76]:
def PLOT_WRONG_PREDICTION(index_selected):
    print(index_selected)
    plt.figure(figsize=(3,3), dpi=120)
    plt.imshow(test_data_x[wrong_index_test[index_selected]][0].asnumpy().astype(np.uint8))
    plt.axis('off')
    plt.title("NN Recognizes as %d .\nReal Answer is %d ." %(y_pred_test[wrong_index_test[index_selected]], test_data_y[wrong_index_test[index_selected]].asscalar()) )
    
print('Total %d wrong predcition. Slide to see the corresponding image.' %len(wrong_index_test))
if len(wrong_index_test)!=0:
    interact(PLOT_WRONG_PREDICTION, index_selected=IntSlider(value=0, min=0, max=len(wrong_index_test)-1));

Total 3 wrong predcition. Slide to see the corresponding image.


interactive(children=(IntSlider(value=0, description='index_selected', max=2), Output()), _dom_classes=('widge…

### Confusion Matrix of Extra Validation Data Set

In [77]:
y_pred_extra_validation = np.zeros(total_extra_validation_data)
count = 0
for X, _ in extra_validation_iter:
    y_pred_extra_validation[count:count+batch_size] = net_RESNET(X.as_in_context(ctx)).asnumpy().argmax(axis=1)
    count += batch_size
wrong_index_extra_validation = np.argwhere(y_pred_extra_validation!=extra_validation_data_y.asnumpy())

In [78]:
result = confusion_matrix(extra_validation_data_y.asnumpy(), y_pred_extra_validation)
print(result)
print('\n')
for i in range(6):
    print("Accuracy of %d is %.2f %%." %(i, 100*result[i][i]/result[i].sum()))

[[ 2  0  0  0  0  0]
 [ 0  4  2  0  0  0]
 [ 0  0  5  1  0  0]
 [ 0  4  0 21  2  0]
 [ 0  0  0  1  4  0]
 [ 0  0  0  0  2  8]]


Accuracy of 0 is 100.00 %.
Accuracy of 1 is 66.67 %.
Accuracy of 2 is 83.33 %.
Accuracy of 3 is 77.78 %.
Accuracy of 4 is 80.00 %.
Accuracy of 5 is 80.00 %.


In [79]:
def PLOT_WRONG_PREDICTION(index_selected):
    print(index_selected)
    plt.figure(figsize=(3,3), dpi=120)
    plt.imshow(extra_validation_data_x[wrong_index_extra_validation[index_selected]][0].asnumpy().astype(np.uint8))
    plt.axis('off')
    plt.title("NN Recognizes as %d .\nReal Answer is %d ." %(y_pred_extra_validation[wrong_index_extra_validation[index_selected]], extra_validation_data_y[wrong_index_extra_validation[index_selected]].asscalar()) )
    
print('Total %d wrong predcition. Slide to see the corresponding image.' %len(wrong_index_extra_validation))
interact(PLOT_WRONG_PREDICTION, index_selected=IntSlider(value=0, min=0, max=len(wrong_index_extra_validation)-1));

Total 12 wrong predcition. Slide to see the corresponding image.


interactive(children=(IntSlider(value=0, description='index_selected', max=11), Output()), _dom_classes=('widg…

### Confusion Matrix of Final Validation Data Set

In [80]:
y_pred_final_validation = np.zeros(total_final_validation_data)
count = 0
for X, _ in final_validation_iter:
    y_pred_final_validation[count:count+batch_size] = net_RESNET(X.as_in_context(ctx)).asnumpy().argmax(axis=1)
    count += batch_size
wrong_index_final_validation = np.argwhere(y_pred_final_validation!=final_validation_data_y.asnumpy())

In [81]:
result = confusion_matrix(final_validation_data_y.asnumpy(), y_pred_final_validation)
print(result)
print('\n')
for i in range(6):
    print("Accuracy of %d is %.2f %%." %(i, 100*result[i][i]/result[i].sum()))

[[59  3  3  0  0  0]
 [ 0 64  2  0  0  1]
 [ 1  2 61  2  0  0]
 [ 0  2  3 64  0  1]
 [ 0  3  0  0 67  1]
 [ 0  2  0  0  3 61]]


Accuracy of 0 is 90.77 %.
Accuracy of 1 is 95.52 %.
Accuracy of 2 is 92.42 %.
Accuracy of 3 is 91.43 %.
Accuracy of 4 is 94.37 %.
Accuracy of 5 is 92.42 %.


In [82]:
def PLOT_WRONG_PREDICTION(index_selected):
    print(index_selected)
    plt.figure(figsize=(3,3), dpi=120)
    plt.imshow(final_validation_data_x[wrong_index_final_validation[index_selected]][0].asnumpy().astype(np.uint8))
    plt.axis('off')
    plt.title("NN Recognizes as %d .\nReal Answer is %d ." %(y_pred_final_validation[wrong_index_final_validation[index_selected]], final_validation_data_y[wrong_index_final_validation[index_selected]].asscalar()) )
    
print('Total %d wrong predcition. Slide to see the corresponding image.' %len(wrong_index_final_validation))
interact(PLOT_WRONG_PREDICTION, index_selected=IntSlider(value=0, min=0, max=len(wrong_index_final_validation)-1));

Total 29 wrong predcition. Slide to see the corresponding image.


interactive(children=(IntSlider(value=0, description='index_selected', max=28), Output()), _dom_classes=('widg…